In [1]:
import pandas as pd
import os
import git

In [2]:
def get_git_root(path):
    git_repo = git.Repo(path, search_parent_directories=True)
    git_root = git_repo.git.rev_parse("--show-toplevel")
    return git_root

txn_date_col_name = "txn_date"
txn_description_col_name = "txn_description"
txn_amount_col_name = "txn_amount"
txn_category_col_name = "category"
txn_subcat_col_name = "sub_category"
processed_txn_filename = "categorized_txn.csv"
bank_col_name = "bank"
id_col_name = "unique_id"
txn_strftime_format = "%Y-%m-%d"
year_col_name = 'year'
month_col_name = 'month'
txn_amt_sum_col_name = 'txn_amt_sum'
num_txns_col_name = 'num_txns'

data_root_dir_append = "data"
data_raw_dir_append = "data/raw"
data_interim_dir_append = "data/interim"
data_processed_dir_append = "data/processed"

processed_txn_filename = "categorized_txn.csv"

In [3]:
git_root_path = get_git_root(os.getcwd())
interim_data_path = os.path.join(git_root_path, data_interim_dir_append)
interim_txn_path = os.path.join(interim_data_path, processed_txn_filename)

interim_txn_df = pd.read_csv(interim_txn_path, index_col= id_col_name)
interim_txn_df[txn_date_col_name] = pd.to_datetime(interim_txn_df[txn_date_col_name], format=txn_strftime_format)
#interim_txn_df[txn_date_col_name] = pd.to_datetime(interim_txn_df[txn_date_col_name], format="%m/%d/%y")

In [4]:
processed_txn_df = interim_txn_df.copy()

In [5]:
processed_txn_df['year'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].year
    , axis = 1
)

processed_txn_df['month'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].month
    , axis = 1
)

processed_txn_df['day'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].day
    , axis = 1
)

processed_txn_df['weekday'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].strftime("%A")
    , axis = 1
)

# get expenses to be positive
processed_txn_df[txn_amount_col_name] = processed_txn_df.apply(
    lambda x: 
    -1 * x[txn_amount_col_name] if x[txn_category_col_name] != 'Salary' else x[txn_amount_col_name]
    , axis = 1
)

In [6]:
# cat_detail_df = processed_txn_df.groupby([txn_category_col_name, 'year', 'month']).agg(
#     sum_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=sum),
#     num_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=pd.Series.nunique)
# ).reset_index()

subcat_detail_df = processed_txn_df.groupby([txn_category_col_name, txn_subcat_col_name, 'year', 'month']).agg(
    txn_amt_sum = pd.NamedAgg(column = txn_amount_col_name, aggfunc=sum),
    num_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=pd.Series.nunique)
).reset_index()

full_cat_detail_df = subcat_detail_df.groupby([txn_category_col_name, year_col_name, month_col_name]).agg(
    txn_amt_sum = pd.NamedAgg(column = txn_amt_sum_col_name, aggfunc=sum),
    num_txns = pd.NamedAgg(column = num_txns_col_name, aggfunc=sum),
).reset_index()

cat_detail_df_expenses = subcat_detail_df[~subcat_detail_df[txn_subcat_col_name]\
    .isin(['Credit Card Payment'])].reset_index()

cat_detail_df_expenses = cat_detail_df_expenses[~cat_detail_df_expenses[txn_category_col_name]\
    .isin(['Investments', 'Income'])]\
    .groupby([txn_category_col_name, year_col_name, month_col_name])\
    .agg(
        txn_amt_sum = pd.NamedAgg(column = txn_amt_sum_col_name, aggfunc=sum),
        num_txns = pd.NamedAgg(column = num_txns_col_name, aggfunc=pd.Series.nunique)
    ).reset_index()

subcat_detail_df['avg_per_txn'] = subcat_detail_df[txn_amt_sum_col_name] / subcat_detail_df[num_txns_col_name]
cat_detail_df_expenses['avg_per_txn'] = cat_detail_df_expenses[txn_amt_sum_col_name] / cat_detail_df_expenses[num_txns_col_name]
full_cat_detail_df['avg_per_txn'] = full_cat_detail_df[txn_amt_sum_col_name] / full_cat_detail_df[num_txns_col_name]

In [7]:
subcat_detail_df.head(15)

,category,sub_category,year,month,txn_amt_sum,num_txns,avg_per_txn
0,Auto and Transport,Gas,2019,8,99.54,2.0,49.770000
1,Auto and Transport,Gas,2019,9,187.68,6.0,31.280000
2,Auto and Transport,Gas,2019,10,359.67,7.0,51.381429
3,Auto and Transport,Gas,2019,11,273.18,5.0,54.636000
4,Auto and Transport,Gas,2019,12,194.96,4.0,48.740000
5,Auto and Transport,Insurance,2019,10,15.00,1.0,15.000000
6,Auto and Transport,Miscellaneous,2019,11,29.40,1.0,29.400000
7,Auto and Transport,Parking,2019,8,16.50,1.0,16.500000
8,Auto and Transport,Parking,2019,9,5.00,2.0,2.500000
9,Auto and Transport,Parking,2019,10,3.25,2.0,1.625000


In [22]:
def fill_missing_months(df, col_name_to_fill, category = None):
    
    df_columns = df.columns
    years = df[year_col_name].unique()
    months = df[month_col_name].unique()
    
    if category is not None:
        df = df[df[txn_category_col_name] == category]

    for year in years:

        year_df = df[df[year_col_name] == year]
        column_val_array = year_df[col_name_to_fill].unique()

        for col_val in column_val_array:

            col_val_df = year_df[year_df[col_name_to_fill] == col_val]
            
            for month in months:

                df_subset = col_val_df[(col_val_df[year_col_name] == year) & (col_val_df[month_col_name] == month)] 
                row_exists = True if len(df_subset) > 0 else False

                if row_exists is False:
                    new_row = {
                        year_col_name : year,
                        month_col_name : month,
                        col_name_to_fill : col_val
                    }
                    if category is not None:
                        new_row[txn_category_col_name] = category

                    new_row_full = {
                        col : (0 if col not in new_row.keys() else new_row[col])
                        for col in df_columns
                    }

                    df = df.append(new_row_full, ignore_index=True)
                    
    return df

In [9]:
cat_detail_df_expenses = fill_missing_months(cat_detail_df_expenses, txn_category_col_name)
cat_detail_df_expenses = cat_detail_df_expenses.sort_values([txn_category_col_name, year_col_name, month_col_name]).reset_index(drop=True)

In [23]:
subcat_detail_df_full = pd.DataFrame()
for cat in subcat_detail_df[txn_category_col_name].unique():
    
    #subcat_detail_df_subset = subcat_detail_df[subcat_detail_df[txn_category_col_name] == cat]
    subcat_detail_df_subset = fill_missing_months(subcat_detail_df, txn_subcat_col_name, cat)
    subcat_detail_df_full = subcat_detail_df_full.append(subcat_detail_df_subset, ignore_index=True)
    
subcat_detail_df_full = subcat_detail_df_full.sort_values([txn_category_col_name, txn_subcat_col_name, year_col_name, month_col_name]).reset_index(drop=True)

In [24]:
subcat_detail_df_full.to_csv('subset_detail.csv', index=False)

In [ ]:
cat_detail_df_expenses.pivot(index=month_col_name, columns=txn_category_col_name, values=txn_amt_sum_col_name).plot(kind='bar')

In [13]:
subcat_detail_df[subcat_detail_df[txn_category_col_name] == 'Auto and Transport']

,category,sub_category,year,month,txn_amt_sum,num_txns,avg_per_txn
0,Auto and Transport,Gas,2019,8,99.54,2.0,49.770000
1,Auto and Transport,Gas,2019,9,187.68,6.0,31.280000
2,Auto and Transport,Gas,2019,10,359.67,7.0,51.381429
3,Auto and Transport,Gas,2019,11,273.18,5.0,54.636000
4,Auto and Transport,Gas,2019,12,194.96,4.0,48.740000
5,Auto and Transport,Insurance,2019,10,15.00,1.0,15.000000
6,Auto and Transport,Miscellaneous,2019,11,29.40,1.0,29.400000
7,Auto and Transport,Parking,2019,8,16.50,1.0,16.500000
8,Auto and Transport,Parking,2019,9,5.00,2.0,2.500000
9,Auto and Transport,Parking,2019,10,3.25,2.0,1.625000
